# Requirements

In [1]:
!uv pip install --system huggingface_hub accelerate openai pyngrok

Using Python 3.13.5 environment at: /opt/python
Resolved 52 packages in 488ms                                        
Prepared 4 packages in 284ms                                             
Uninstalled 1 package in 3ms
░░░░░░░░░░░░░░░░░░░░ [0/28] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
██░░░░░░░░░░░░░░░░░░ [3/28] pyngrok==7.3.0                                      warning: The module `nvidia` is provided by more than one package, which causes an install race condition and can result in a broken module. Consider removing your dependency on either `nvidia-nvjitlink-cu12` (v12.8.93) or `nvidia-cuda-cupti-cu12` (v12.8.90).
███░░░░░░░░░░░░░░░░░ [5/28] filel

In [2]:
!uv pip install --system git+https://github.com/huggingface/transformers.git

Using Python 3.13.5 environment at: /opt/python
Resolved 18 packages in 6.47s                                        
Prepared 1 package in 4.84s                                              
░░░░░░░░░░░░░░░░░░░░ [0/2] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 2 packages in 517ms                               
 + tokenizers==0.21.4
 + transformers==4.56.0.dev0 (from git+https://github.com/huggingface/transformers.git@085e02383c3d74a4e18e7ef8404c281b3e8ccfb5)


In [3]:
import torch
print("PyTorch:")
print("PyTorch is working with CUDA" if torch.cuda.is_available() else "Error! It is not working correctly")
print("The GPU model is: "+ torch.cuda.get_device_name(0))

PyTorch:
PyTorch is working with CUDA
The GPU model is: Tesla T4


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

# Run transformers serve

In [ ]:
# terminal 1
!transformers serve

In [4]:
import requests
requests.get("http://localhost:8000/v1/models").json()

{'object': 'list',
 'data': [{'id': 'Menlo/Jan-nano',
   'object': 'model',
   'created': 1749539461.0,
   'owned_by': 'Menlo'},
  {'id': 'Menlo/Jan-nano-128k',
   'object': 'model',
   'created': 1750831938.0,
   'owned_by': 'Menlo'},
  {'id': 'Qwen/Qwen2.5-0.5B-Instruct',
   'object': 'model',
   'created': 1726487566.0,
   'owned_by': 'Qwen'},
  {'id': 'Qwen/Qwen2.5-3B-Instruct',
   'object': 'model',
   'created': 1726582132.0,
   'owned_by': 'Qwen'},
  {'id': 'Qwen/Qwen2.5-7B-Instruct',
   'object': 'model',
   'created': 1726487740.0,
   'owned_by': 'Qwen'},
  {'id': 'Qwen/Qwen2.5-14B-Instruct',
   'object': 'model',
   'created': 1726487770.0,
   'owned_by': 'Qwen'},
  {'id': 'meta-llama/Llama-3.1-8B-Instruct',
   'object': 'model',
   'created': 1721292960.0,
   'owned_by': 'meta-llama'},
  {'id': 'meta-llama/Llama-3.2-1B-Instruct',
   'object': 'model',
   'created': 1726672367.0,
   'owned_by': 'meta-llama'},
  {'id': 'meta-llama/Llama-3.3-70B-Instruct',
   'object': 'model',

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",  # Local Ollama API
    api_key="dummy_string"                       # Dummy key
)

response = client.chat.completions.create(
    model="Menlo/Jan-nano", #meta-llama/Llama-3.2-1B-Instruct
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain what MXFP4 quantization is."}
    ],
    stream=True
)

for chunk in response:
    token = chunk.choices[0].delta.content
    if token:
        print(token, end='')

MXFP4 quantization is a type of **quantization method** used in **machine learning**, particularly in **neural network training and inference**. It is a **post-training quantization** technique that reduces the precision of the model's weights and activations from **32-bit floating-point (FP32)** to **4-bit integers (INT4)**, which can significantly reduce the model's size and improve inference speed while maintaining acceptable accuracy.

### Key Concepts:

1. **Quantization**:
   - Quantization is the process of reducing the precision of numerical values in a model to make it more efficient in terms of memory and computation.
   - In machine learning, this is often done to enable **model compression**, **acceleration**, and **deployment on edge devices** (e.g., mobile phones, IoT devices).

2. **MXFP4**:
   - **MXFP4** stands for **Mixed-Exponent FP4** or **Mixed-Exponent 4-bit Floating Point**.
   - It is a **4-bit floating-point format** that uses a **mixed exponent** scheme to rep

In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "..."

port = "8000"
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

 * ngrok tunnel "https://333fecb39890.ngrok-free.app" -> "http://127.0.0.1:8000"                    
